# Probabilistic classification algorithms

In this tutorial we will implement a Naive Bayes classifier for text classification. More specifically, the aim of the classifier is to predict whether a given product review is positive or negative.

We will represent documents (i.e. product reviews) as vectors of binary features. For each [unigram or bigram](https://en.wikipedia.org/wiki/N-gram) that appears in at least one document in our training dataset we introduce a binary feature. For a given document a feature is 1 if the documents contains the corresponding unigram/bigram, and it is 0 otherwise.

The datasets are contained in four files ('test.negative', 'test.positive', 'train.negative', 'train.positive'). The files correspond to product reviews from the test and train datasets labelled positively or negatively. For example, the file 'test.negative' contains negative product reviews from the test dataset.

The reviews are preprocessed and expressed as a list of unigrams and bigrams without duplications (for a bigram (w1,w2), the corresponding record is w1__w2). Each review is represented as a single line and features (unigrams and bigrams) extracted from that review are listed space delimited. Take a moment to inspect the content of the dataset files.

In [1]:
import numpy as np

1. First, we read the training data, compute feature statistics, and store it in a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) $\texttt{featureStat}$, where $\texttt{featureStat[feature][classLabel]}$ is equal to the number of documents in the class $\texttt{classLabel}$ that contain $\texttt{feature}$. We also compute the total number of positive train instances and the total number of negative train instances.

In [2]:
def count(featureStat, fname, classLabel):
    numInstances = 0
    with open(fname) as file:
        for line in file:
            numInstances += 1
            for feature in line.strip().split():
                if feature not in featureStat:
                    featureStat[feature] = {0:0, 1:0}
                featureStat[feature][classLabel] += 1
    return numInstances

In [3]:
featureStat = {}
numPositiveInstances = count(featureStat, "train.positive", 1)
numNegativeInstances = count(featureStat, "train.negative", 0)

2. Now, compute the conditional probabilities $P(w_i = 1 | C = c)$, i.e. probability of specific feature $w_i$ to be present in a document from class $c$. Use Laplace smooting to avoid zero probabilities.

In [4]:
def laplace_smoothing(featureStat, pos_total, neg_total):
    conditionalProb = {}
    for feature in featureStat:
        conditionalProb[feature] = {0:0, 1:0} 
        
        # compute smoothed conditional probability of P(feature=1 | C = positive)
        conditionalProb[feature][1] = (featureStat[feature][1]+1)/(pos_total+2)
        
        # compute smoothed conditional probability of P(feature=1 | C=negative)
        conditionalProb[feature][0] = (featureStat[feature][0]+1)/(neg_total+2)
        
    return conditionalProb

In [5]:
# Compute Laplace smoothed conditional probabilities P(feat | classLabel) 
conditionalProb = laplace_smoothing(featureStat, numPositiveInstances, numNegativeInstances)

3. Implement a Naive Bayes classfier function that predicts whether a given document belongs to the positive or the negative class. To avoid problems of very small numbers, instead of computing $P(C=c) \cdot \prod_i P(w_i = a_i| C=c)$ cosider computing the $\log$ of that.

In [6]:
def naive_bayes_predict(conditionalProb, document, pos_total, neg_total):
    # Compute log of prior class probabilities
    pos_prior = np.log(pos_total/(pos_total + neg_total))
    neg_prior = np.log(neg_total/(pos_total + neg_total))
    
    # Compute log of likelihood P(document | C) for both positive and negative classes
    pos_likelihood = neg_likelihood = 0

    for feature in conditionalProb:
        if feature in document:
            pos_likelihood += np.log(conditionalProb[feature][1])
            neg_likelihood += np.log(conditionalProb[feature][0])
        else:
            pos_likelihood += np.log(1 - conditionalProb[feature][1])
            neg_likelihood += np.log(1 - conditionalProb[feature][0])           
            
    # Compute log of posterior
    pos_posterior = pos_prior + pos_likelihood
    neg_posterior = neg_prior + neg_likelihood

    return 1 if pos_posterior > neg_posterior else 0

4. Let's now read the test dataset from the files

In [7]:
def getInstances(fname, classLabel):
    data = []
    with open(fname) as file:
        for line in file:
            data.append((classLabel, line.strip().split()))
    return data

# Read test data
test_data = getInstances("test.positive", 1)
test_data.extend(getInstances("test.negative", 0))

4. Evaluate accuracy of the Naive Bayes algorithm on the test data

In [8]:
# Compute and output accuracy    
corrects = 0
for (classLabel, document) in test_data:
    if classLabel == naive_bayes_predict(conditionalProb, document, numPositiveInstances, numNegativeInstances):
        corrects += 1
print("Accuracy =", float(corrects) / float(len(test_data)))

Accuracy = 0.8190954773869347


5. Modify the code and see what happens if we do not use Laplace smoothing